In [1]:
import pandas as pd
import glob
import os
from json import loads, dumps
import json



#combining all the daily gamelogs csv files pulled
path = 'filepath' 
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

dfall = pd.concat(li, axis=0, ignore_index=True)

In [2]:
dfall['GAME_DATE']=dfall['GAME_DATE'].apply(pd.to_datetime)

In [3]:
dfallsort=dfall.sort_values(by=['GAME_DATE','GAME_ID']).reset_index(drop=True)

In [4]:
dfgamenum = dfallsort.copy()

In [5]:
dfgamenum

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,POSS,PIE,GAME_DATE,PTS,OPP_TEAM_ID,OPP_PTS,OPP_POSS,OPP_TEAM_NAME,OPP_TEAM_ABBERVIATION,OPP_TEAM_CITY
0,22400061,1610612752,Knicks,NYK,New York,240.000000:00,118.4,119.8,144.2,145.1,...,91,0.419,2024-10-22,109,1610612738,132,91,Celtics,BOS,Boston
1,22400061,1610612738,Celtics,BOS,Boston,240.000000:00,144.2,145.1,118.4,119.8,...,91,0.581,2024-10-22,132,1610612752,109,91,Knicks,NYK,New York
2,22400062,1610612747,Lakers,LAL,Los Angeles,240.000000:00,112.2,115.8,102.1,109.6,...,95,0.575,2024-10-22,110,1610612750,103,94,Timberwolves,MIN,Minnesota
3,22400062,1610612750,Timberwolves,MIN,Minnesota,240.000000:00,102.1,109.6,112.2,115.8,...,94,0.425,2024-10-22,103,1610612747,110,95,Lakers,LAL,Los Angeles
4,22400063,1610612754,Pacers,IND,Indiana,240.000000:00,112.7,113.9,106.5,110.1,...,101,0.586,2024-10-23,115,1610612765,109,99,Pistons,DET,Detroit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,22400424,1610612743,Nuggets,DEN,Denver,240.000000:00,126.3,128.6,138.4,140.6,...,105,0.463,2024-12-27,135,1610612739,149,106,Cavaliers,CLE,Cleveland
908,22400425,1610612742,Mavericks,DAL,Dallas,240.000000:00,102.6,106.5,93.8,95.7,...,92,0.559,2024-12-27,98,1610612756,89,93,Suns,PHX,Phoenix
909,22400425,1610612756,Suns,PHX,Phoenix,240.000000:00,93.8,95.7,102.6,106.5,...,93,0.441,2024-12-27,89,1610612742,98,92,Mavericks,DAL,Dallas
910,22400426,1610612746,Clippers,LAC,LA,240.000000:00,103.4,104.1,93.1,94.8,...,98,0.545,2024-12-27,102,1610612744,92,97,Warriors,GSW,Golden State


In [6]:
dfgamenum['GAME_NO']=dfgamenum.groupby(['TEAM_ID'])['GAME_DATE'].cumcount() + 1

In [7]:
dfgamenum=dfgamenum.copy()
dfdata=dfgamenum

In [8]:
offptsvar = dfdata.groupby('TEAM_ID')['PTS'].expanding().sum().reset_index(0,drop=True)
offposvar = dfdata.groupby('TEAM_ID')['POSS'].expanding().sum().reset_index(0,drop=True)

defptsvar = dfdata.groupby('TEAM_ID')['OPP_PTS'].expanding().sum().reset_index(0,drop=True)
defposvar = dfdata.groupby('TEAM_ID')['OPP_POSS'].expanding().sum().reset_index(0,drop=True)

In [9]:
dfdata['STD_OFF_RATING']=(offptsvar/offposvar)*100
dfdata['STD_OFF_RATING']=dfdata['STD_OFF_RATING'].round(1)

dfdata['STD_DEF_RATING']=(defptsvar/defposvar)*100
dfdata['STD_DEF_RATING']=dfdata['STD_DEF_RATING'].round(1)

dfdata['STD_NET_RATING']=dfdata['STD_OFF_RATING']-dfdata['STD_DEF_RATING']

In [10]:
#Creating JSON to reference for last N Games
dfarray = dfdata[["TEAM_CITY","GAME_NO","PTS","POSS","OPP_PTS","OPP_POSS"]]


dflastn=dfarray.groupby(['TEAM_CITY'])[['TEAM_CITY',"GAME_NO","PTS","POSS","OPP_PTS","OPP_POSS"]].apply(lambda x: x.to_dict('records')).reset_index(name="TEAMS")
dfarraygrouplastn=dflastn[["TEAMS"]]


In [11]:
dfarraygroupjsonlastn= dfarraygrouplastn.to_json(orient="records")


In [12]:
parsedjsonlastn = json.loads(dfarraygroupjsonlastn)


In [13]:
dflg=dfdata[['GAME_NO','PTS','POSS']]

In [14]:
dflg=dflg.groupby("GAME_NO").sum().reset_index()

In [15]:
dflg['PTS']=dflg['PTS'].cumsum()
dflg['POSS']=dflg['POSS'].cumsum()

In [16]:
dflg=dflg.copy()

In [17]:
dflg['RTG']=(dflg['PTS']/dflg['POSS'])*100


In [18]:
#Rankign calcuation

dfrankingdata=dfgamenum
dfrankingdata['STD_OFF_RATING']=(offptsvar/offposvar)*100
dfrankingdata['STD_OFF_RATING']=dfrankingdata['STD_OFF_RATING']

dfrankingdata['STD_DEF_RATING']=(defptsvar/defposvar)*100
dfrankingdata['STD_DEF_RATING']=dfrankingdata['STD_DEF_RATING']

dfrankingdata['STD_NET_RATING']=dfrankingdata['STD_OFF_RATING']-dfrankingdata['STD_DEF_RATING']

dflastgame = dfrankingdata.sort_values('GAME_NO').groupby('TEAM_CITY').tail(1).reset_index(drop="True")

dfrank = dflastgame[["TEAM_CITY","STD_OFF_RATING","STD_DEF_RATING","STD_NET_RATING"]]

dfrank=dfrank.copy()
dfrank['DEF_RANKING'] = dfrank['STD_DEF_RATING'].rank().astype(int)
dfrank['OFF_RANKING'] = dfrank['STD_OFF_RATING'].rank(ascending=False).astype(int)
dfrank['NET_RANKING'] = dfrank['STD_NET_RATING'].rank(ascending=False).astype(int)

In [19]:
#Creating Json for Team Rank for Off and Def Rating
dfrankcols = dfrank[['TEAM_CITY','OFF_RANKING','DEF_RANKING','NET_RANKING']]
dfrankcols=dfrankcols.copy()
dfrankcols.sort_values(by='TEAM_CITY',inplace=True)

In [20]:
dfrankcols.set_index('TEAM_CITY', inplace=True)
dfrankjson=dfrankcols.to_json(orient="index")
dfrankparsed = json.loads(dfrankjson)

In [21]:
dfdatajsoncols = dfdata[['GAME_ID', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CITY',
                         'POSS', 'GAME_DATE', 'PTS', 'OPP_TEAM_ID', 
                         'OPP_PTS', 'OPP_POSS', 'OPP_TEAM_NAME', 'OPP_TEAM_ABBERVIATION',
                         'OPP_TEAM_CITY', 'GAME_NO', 'STD_OFF_RATING', 'STD_DEF_RATING','STD_NET_RATING']]

In [23]:
dfdatajsoncols=dfdatajsoncols.copy()

dfdatajsoncols['GAME_DATE'] = dfdatajsoncols['GAME_DATE'].astype(str)

In [24]:
cols=['GAME_ID','POSS', 'GAME_DATE', 'PTS', 'OPP_TEAM_ID', 
                         'OPP_PTS', 'OPP_POSS', 'OPP_TEAM_NAME', 'OPP_TEAM_ABBERVIATION',
                         'OPP_TEAM_CITY', 'GAME_NO', 'STD_OFF_RATING', 'STD_DEF_RATING','STD_NET_RATING']

In [25]:
dfb=dfdatajsoncols.groupby(['TEAM_NAME','TEAM_ID','TEAM_CITY','TEAM_ABBREVIATION'])[cols].apply(lambda x: x.to_dict('records')).reset_index(name='GAME_INFO')

In [26]:
dflg2= dflg[['GAME_NO','RTG']]

In [27]:
jsonlgavgdata=dflg2.to_json(orient="records")

In [28]:
dfb.set_index('TEAM_CITY', inplace=True)

In [29]:
jsondata=dfb.to_json(orient="index")


In [30]:
parsed = json.loads(jsondata)


In [31]:


parsedlgavg = json.loads(jsonlgavgdata)


In [32]:
#combine jsons into single JSON to reference
combined = [parsedlgavg, parsed,parsedjsonlastn ,dfrankparsed]

In [33]:

with open('filepath', 'w', encoding='utf-8') as f:
    json.dump(combined, f, ensure_ascii=False)